# Pangu mini model demo
---
``python version = 3.11``

> [08/02/2024]: En este notebook se hace intenta probar el modelo [Pangu Weather mini](https://github.com/rudolfmard/Pangu-Weather-mini/tree/main).

In [1]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install timm
# !pip install pygrib # En linux
# !conda install conda-forge::pygrib # En windows use -y flag

In [1]:
import sys
import os
import importlib
# Importamos paquete local
sys.path.append(os.path.dirname(os.getcwd()))

In [28]:
import xarray as xr
import torch
import 

In [2]:
from trainer import Trainer
from model import WeatherModel
import data_handler as dh
import torch
from torch.distributed import init_process_group, destroy_process_group

In [4]:
# Esta debe usarse para configurar el uso de GPU
"""
if execution_mode == "single_gpu":
    print("Training on single GPU.")
elif execution_mode == "multi_gpu":
    print("Training on multiple GPUs.")
    init_process_group(backend="nccl")
else:
    raise ValueError("Invalid execution mode. Valid values are 'single_gpu' or 'multi_gpu'")"""

# This environment variable tells PyTorch CUDA allocator not to split memory blocks larger than certain size.
# Mitigates GPU memory fragmentation and allows the training of the full original model to fit onto one GPU.
"""os.environ["PYTORCH_CUDA_ALLOC_CONF"] = 'max_split_size_mb:1024'"""

# Path to checkpoint.pt to continue training from that checkpoint,
# if checkpoint.pt does not exist training starts from scratch.
"""checkpoint_path="checkpoint.pt" """

'checkpoint_path="checkpoint.pt" '

In [3]:

"""
Training parameters:
learning_rate (float):   Learning rate of the training, 5e-4 in original Pangu-Weather.
max_epochs (int):        Maximum number of epochs for training, 100 in original Pangu-Weather.
save_every (int):        Saves a checkpoint every save_every epoch.
batch_size (int):        Batch size of the training data, 1 in original Pangu-Weather.
"""
learning_rate = 5e-4
max_epochs = 10
save_every = 2
batch_size = 1

"""
Model parameters:
C (int):                Dimensionality of patch embedding of the tokens. 192 in original Pangu-Weather. Make sure C is divisible by n_heads.
depth (list[int]):      List with length of 4, defines the number of transformer blocks in each 4 EarthSpecificLayers. [2,6,6,2] in original Pangu-Weather.
n_heads (list[int]):    List with length of 4, defines the number of heads in transformer blocks of each 4 EarthSpecificLayers. [6, 12, 12, 6] in original Pangu-Weather.
D (int):                Dimensionality multiplier of hidden layer in transformer MLP. 4 in original Pangu-Weather.
"""
C = 192
depth = [2, 6, 6, 2]
n_heads = [6, 12, 12, 6]
D = 4

# Create a model object:
model = WeatherModel(C, depth, n_heads, D, batch_size, log_GPU_mem=True)

c:\Users\gcuervo\miniconda3\envs\art1_pyenv\Lib\site-packages\torch\functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[09/02/2024]
`C:\Users\gcuervo\miniconda3\envs\art1_pyenv\Lib\site-packages\timm\layers\drop.py`

line 170: 
```python
return f'drop_prob={self.drop_prob} ====> return f'drop_prob={round(self.drop_prob.item(),3):0.3f}
```
fix: ``.item()`` method added

In [4]:
model

WeatherModel(
  (input_layer): PatchEmbedding(
    (conv_air): Conv3d(5, 192, kernel_size=(2, 4, 4), stride=(2, 4, 4))
    (conv_surface): Conv2d(7, 192, kernel_size=(4, 4), stride=(4, 4))
  )
  (layer1): EarthSpecificLayer(
    (blocks): ModuleList(
      (0): EarthSpecificBlock(
        (attention): EarthAttention3D(
          (linear_qkv): Linear(in_features=192, out_features=576, bias=True)
          (linear): Linear(in_features=192, out_features=192, bias=True)
          (dropout): Dropout(p=0, inplace=False)
        )
        (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (feedforward): MLP(
          (linear1): Linear(in_features=192, out_features=768, bias=True)
          (linear2): Linear(in_features=768, out_features=192, bias=True)
          (drop): Dropout(p=0, inplace=False)
        )
        (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (drop_path): DropPath(drop_prob=0.000)
      )
      (1): EarthSpecificBlock(
        (a

In [5]:
dh.__dict__

{'__name__': 'data_handler',
 '__doc__': None,
 '__package__': '',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x1237b864150>,
 '__spec__': ModuleSpec(name='data_handler', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001237B864150>, origin='c:\\Users\\gcuervo\\OneDrive - Universidad de Las Palmas de Gran Canaria\\Documents\\Doctorado\\PhD_repo\\models\\PanguWeather_mini_model\\data_handler.py'),
 '__file__': 'c:\\Users\\gcuervo\\OneDrive - Universidad de Las Palmas de Gran Canaria\\Documents\\Doctorado\\PhD_repo\\models\\PanguWeather_mini_model\\data_handler.py',
 '__cached__': 'c:\\Users\\gcuervo\\OneDrive - Universidad de Las Palmas de Gran Canaria\\Documents\\Doctorado\\PhD_repo\\models\\PanguWeather_mini_model\\__pycache__\\data_handler.cpython-311.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, types, exceptions, and other objects.\n\nThis module provides direct access to all 'built-in'\nidentifiers of Pyth

In [2]:
ERA5_air = xr.load_dataset(
    r"C:\Users\gcuervo\OneDrive - Universidad de Las Palmas de Gran Canaria\Documents\Doctorado\DB\2010-2023_01_10-6h-64x32_vars_filtered.nc")
ERA5_air

<xarray.Dataset>
Dimensions:              (time: 19032, level: 13, longitude: 64, latitude: 32)
Coordinates:
  * latitude             (latitude) float64 -87.19 -81.56 -75.94 ... 81.56 87.19
  * level                (level) int64 50 100 150 200 250 ... 700 850 925 1000
  * longitude            (longitude) float64 0.0 5.625 11.25 ... 348.8 354.4
  * time                 (time) datetime64[ns] 2010-01-01 ... 2023-01-10T18:0...
Data variables:
    geopotential         (time, level, longitude, latitude) float32 2.024e+05...
    temperature          (time, level, longitude, latitude) float32 238.1 ......
    specific_humidity    (time, level, longitude, latitude) float32 2.721e-06...
    u_component_of_wind  (time, level, longitude, latitude) float32 -5.926 .....
    v_component_of_wind  (time, level, longitude, latitude) float32 5.229 ......

In [25]:
i_train = round(ERA5_air.time.values.size * 0.7)
ERA5_air_train = ERA5_air.isel(time=slice(0, i_train))
ERA5_air_val = ERA5_air.isel(time=slice(i_train, -1))

In [3]:
ERA5_surf = xr.load_dataset(
    r"C:\Users\gcuervo\OneDrive - Universidad de Las Palmas de Gran Canaria\Documents\Doctorado\DB\2010-2023_01_10-6h-64x32_surfvars_filtered.nc")
ERA5_surf

<xarray.Dataset>
Dimensions:                  (time: 19032, longitude: 64, latitude: 32)
Coordinates:
  * latitude                 (latitude) float64 -87.19 -81.56 ... 81.56 87.19
  * longitude                (longitude) float64 0.0 5.625 11.25 ... 348.8 354.4
  * time                     (time) datetime64[ns] 2010-01-01 ... 2023-01-10T...
Data variables:
    2m_temperature           (time, longitude, latitude) float32 247.5 ... 263.3
    surface_pressure         (time, longitude, latitude) float32 7.122e+04 .....
    10m_wind_speed           (time, longitude, latitude) float32 6.803 ... 10.89
    total_precipitation_6hr  (time, longitude, latitude) float32 9.716e-09 .....

In [26]:
i_train = round(ERA5_surf.time.values.size * 0.7)
ERA5_surf_train = ERA5_surf.isel(time=slice(0, i_train))
ERA5_surf_val = ERA5_surf.isel(time=slice(i_train, -1))

In [53]:
test

<xarray.Dataset>
Dimensions:              (batch: 1, time: 13322, level: 13, longitude: 64,
                          latitude: 32)
Coordinates:
  * latitude             (latitude) float64 -87.19 -81.56 -75.94 ... 81.56 87.19
  * level                (level) int64 50 100 150 200 250 ... 700 850 925 1000
  * longitude            (longitude) float64 0.0 5.625 11.25 ... 348.8 354.4
  * time                 (time) datetime64[ns] 2010-01-01 ... 2019-02-13T06:0...
Dimensions without coordinates: batch
Data variables:
    geopotential         (batch, time, level, longitude, latitude) float32 dask.array<chunksize=(1, 14, 13, 64, 32), meta=np.ndarray>
    temperature          (batch, time, level, longitude, latitude) float32 dask.array<chunksize=(1, 14, 13, 64, 32), meta=np.ndarray>
    specific_humidity    (batch, time, level, longitude, latitude) float32 dask.array<chunksize=(1, 14, 13, 64, 32), meta=np.ndarray>
    u_component_of_wind  (batch, time, level, longitude, latitude) float32 dask.array<chunksize=(1, 14, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind  (batch, time, level, longitude, latitude) float32 dask.array<chunksize=(1, 14, 13, 64, 32), meta=np.ndarray>

In [59]:
test = ERA5_air_train.chunk({'time': 14})  # .expand_dims('batch')
test

<xarray.Dataset>
Dimensions:              (time: 13322, level: 13, longitude: 64, latitude: 32)
Coordinates:
  * latitude             (latitude) float64 -87.19 -81.56 -75.94 ... 81.56 87.19
  * level                (level) int64 50 100 150 200 250 ... 700 850 925 1000
  * longitude            (longitude) float64 0.0 5.625 11.25 ... 348.8 354.4
  * time                 (time) datetime64[ns] 2010-01-01 ... 2019-02-13T06:0...
Data variables:
    geopotential         (time, level, longitude, latitude) float32 dask.array<chunksize=(14, 13, 64, 32), meta=np.ndarray>
    temperature          (time, level, longitude, latitude) float32 dask.array<chunksize=(14, 13, 64, 32), meta=np.ndarray>
    specific_humidity    (time, level, longitude, latitude) float32 dask.array<chunksize=(14, 13, 64, 32), meta=np.ndarray>
    u_component_of_wind  (time, level, longitude, latitude) float32 dask.array<chunksize=(14, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind  (time, level, longitude, latitude) float32 dask.array<chunksize=(14, 13, 64, 32), meta=np.ndarray>

In [62]:
test.chunk.__sizeof__()

48

In [64]:
batch_coords = xr.DataArray(
    range(test.chunk.__sizeof__()), dims=('batch',), name='batch')
batch_coords

<xarray.DataArray 'batch' (batch: 48)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47])
Dimensions without coordinates: batch

In [70]:
test.coords['batch'] = batch_coords
test.dims.mapping

{'time': 13322, 'level': 13, 'longitude': 64, 'latitude': 32, 'batch': 48}

In [68]:
test.value

AttributeError: 'Dataset' object has no attribute 'value'

In [47]:
xr.DataArray(range(len(test.batch)), dims=('batch',), name='batch')

<xarray.DataArray 'batch' (batch: 1)>
array([0])
Dimensions without coordinates: batch

In [ ]:
hunk_coords = xr.DataArray(range(len(ERA5_air_train.time)), dims=('chunk',), name='chunk')

In [74]:
test.as_numpy()

<xarray.Dataset>
Dimensions:              (time: 13322, level: 13, longitude: 64, latitude: 32,
                          batch: 48)
Coordinates:
  * latitude             (latitude) float64 -87.19 -81.56 -75.94 ... 81.56 87.19
  * level                (level) int64 50 100 150 200 250 ... 700 850 925 1000
  * longitude            (longitude) float64 0.0 5.625 11.25 ... 348.8 354.4
  * time                 (time) datetime64[ns] 2010-01-01 ... 2019-02-13T06:0...
  * batch                (batch) int32 0 1 2 3 4 5 6 7 ... 41 42 43 44 45 46 47
Data variables:
    geopotential         (time, level, longitude, latitude) float32 2.024e+05...
    temperature          (time, level, longitude, latitude) float32 238.1 ......
    specific_humidity    (time, level, longitude, latitude) float32 2.721e-06...
    u_component_of_wind  (time, level, longitude, latitude) float32 -5.926 .....
    v_component_of_wind  (time, level, longitude, latitude) float32 5.229 ......

In [41]:
= list(range(len(ERA5_air_train.batch)))

AttributeError: 'Dataset' object has no attribute 'batch'

In [36]:
# Convertir el objeto xarray a un tensor de PyTorch
ERA5_air_train_tensor = torch.from_numpy(ERA5_air_train.values)

# Guardar el tensor en un archivo .pt
torch.save(ERA5_air_train_tensor,
           r"C:\Users\gcuervo\OneDrive - Universidad de Las Palmas de Gran Canaria\Documents\Doctorado\DB\Pangu_data\ERA5_air_train.pt")

TypeError: expected np.ndarray (got Dataset)

In [14]:

# Create dataloader objects for training and validation data:
train_dataset = dh.WeatherDataset(lead_time=1, air_data_path="../weather_data/air_test.pt", surface_data_path="../weather_data/surface_test.pt")
train_dataloader = dh.prepare_dataloader(train_dataset, batch_size, execution_mode)

# If validation_dataloader is set to None, no validation is performed between epochs.
validation_dataset = dh.WeatherDataset(lead_time=1, air_data_path="../weather_data/air_test_validation.pt", surface_data_path="../weather_data/surface_test_validation.pt")
validation_dataloader = dh.prepare_dataloader(validation_dataset, batch_size, execution_mode)


FileNotFoundError: [Errno 2] No such file or directory: '../weather_data/air_test.pt'

In [ ]:

    # Create loss loss function and optimizer objects:
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=3e-6)
    loss_fn = torch.nn.L1Loss()


In [ ]:

    # Create a trainer object and train the model:
    trainer = Trainer(model, train_dataloader, validation_dataloader, loss_fn, optimizer, max_epochs, save_every, execution_mode, checkpoint_path)
    trainer.train()


In [ ]:

    # Calculate RMSE on a batch of data:
    calculate_RMSE = True
    if calculate_RMSE:
        with torch.no_grad(): 
            device = next(model.parameters()).device
            model.eval()

            # Fetch a batch from validation dataloader:
            data, targets = next(iter(validation_dataloader))

            # Move the data to the same device as the model:
            data_air, data_surface = data
            data_air = data_air.to(device)
            data_surface = data_surface.to(device)

            targets_air, targets_surface = targets
            targets_air = targets_air.to(device)
            targets_surface = targets_surface.to(device)

            # Make prediction with the model:
            output_air, output_surface = model((data_air, data_surface))

            # Calculate RMSE of the predictions on unnormalized data:
            rmse_values = dh.RMSE((dh.unnormalize_data(output_air), dh.unnormalize_data(output_surface)), 
                                (dh.unnormalize_data(targets_air), dh.unnormalize_data(targets_surface)), save=True)

    if execution_mode == "multi_gpu":
        destroy_process_group()